In [1]:
import matplotlib as mpl
import matplotlib.colors as col
import matplotlib.pyplot as plt
import itertools
import numpy as np
import pandas as pd
import copy

## Schelling model

1. Populate a $N \times N$ grid with $n$ colors
2. Let randomly drawn cells move if they are **unsatisfied,** i.e. they have more than $k$ neighbors of not the same color.
3. Repeat for $M$ iterations


In [ ]:
%matplotlib inline

mpl.rcParams.update(mpl.rcParamsDefault)
e = ['#ffffff', 
     '#009FD7',  
     '#E30010',  
     '#f39200', 
     '#528EA5',  
     '#004C64', 
     '#E84932',  
     '#74BCBF',  
     '#fed630',  
     '#75D0F4',  
     '#EBA289']

cmap = col.LinearSegmentedColormap.from_list('', [col.hex2color(color) for color in e], N=len(e))

In [3]:
np.random.RandomState(11235)

In [26]:
class Schelling:
    def __init__(self, N, empty_ratio, similarity_threshold, races=2):
        self.N = N
        self.races = races
        self.empty_ratio = empty_ratio
        self.similarity_threshold = similarity_threshold

        self.prob = [self.empty_ratio] + [(1 - self.empty_ratio)/(self.races) for i in range(self.races)]
        
        self.map = np.random.choice(self.races+1, 
                                     size=(self.N+1)**2, 
                                     replace=True, 
                                     p=self.prob,
                                     ).reshape(self.N+1, self.N+1)
        self.empty_houses = np.argwhere(self.map==0)
        self.houses_by_race = {i: np.argwhere(self.map==i) for i in np.arange(1, self.races+1)}
        self.households = {tuple(k) : self.map[k[0], k[1]] for k in np.argwhere(self.map>0)}

        
    def is_satisfied(self, x):
        # assume empty cells are satisfied
        if x not in self.households.keys():
            satisfied = True
        
        ######
        ## assume the border is satisfied - 
        ## this is just a simplification for the next part
        #####
        elif x[0] in [0, self.N+1] or x[1] in [0, self.N+1]:
            satisfied = True
        
        ######
        ## check the eight neighbors of each household in the interior
        ## x x x 
        ## x H x
        ## x x x
        ######
        else:
            same = len(np.where(np.isin(self.map[x[0]-1:x[0]+2, x[1]-1:x[1]+2],
                                [self.map[x[0],x[1]], 0]))[0])-1

            if same >= 8*self.similarity_threshold:
                satisfied = True
            else: 
                satisfied = False

        return satisfied

    def update(self):
        self.satisfied = np.array([self.is_satisfied(tuple(x)) for x in np.argwhere(self.map>-1)]).reshape(self.N+1, self.N+1)

        unsatisfied = np.random.permutation(np.argwhere(self.satisfied==False))

        available = np.random.permutation(np.concatenate([self.empty_houses, unsatisfied]))

        for k in range(len(unsatisfied)):
            avail = tuple(available[k])
            self.map[avail[0], avail[1]] = self.households[tuple(unsatisfied[k])]

        now_empty = np.delete(available, np.s_[:len(unsatisfied)], 0)

        for k in now_empty:
            self.map[k[0], k[1]] = 0

        self.empty_houses = np.argwhere(self.map==0)

        self.houses_by_race = {i: np.argwhere(self.map==i) for i in np.arange(1, self.races+1)}

        self.households = {tuple(k) : self.map[k[0], k[1]] for k in np.argwhere(self.map>0)}

        self.satisfied = np.array([self.is_satisfied(tuple(x)) for x in np.argwhere(self.map>-1)]).reshape(self.N+1, self.N+1)

        if len(np.argwhere(self.satisfied==False)) == 0:
            print("Simulation stable")

        

In [ ]:
N = 100
races = 2
empty_ratio = .2
similarity_threshold = .6
n_iterations = 10

test = Schelling(N, empty_ratio, similarity_threshold, races)

test.update()

fig, ax = plt.subplots(figsize=(6,6))

plt.imshow(test.map[1:test.N, 1:test.N], cmap=cmap);

ss = np.array([test.is_satisfied(tuple(x)) for x in np.argwhere(test.map>-1)]).reshape(test.N+1, test.N+1)

print("Initial State - Unsatisfied: {}".format(len(np.argwhere(ss==False))))
plt.show()

for i in range(50):
    test.update()
    
    if i % 5 == 4:

        ss = np.array([test.is_satisfied(tuple(x)) for x in np.argwhere(test.map>-1)]).reshape(test.N+1, test.N+1)

        print("Step: {} - Unsatisfied: {}".format(i+1, len(np.argwhere(ss==False))))

        fig, ax = plt.subplots(figsize=(6,6))

        plt.imshow(test.map[1:test.N, 1:test.N], cmap=cmap);

        plt.show()

In [27]:
N = 5
races = 2
empty_ratio = .2
similarity_threshold = .6
n_iterations = 10

test = Schelling(N, empty_ratio, similarity_threshold, races)

test.houses_by_race

{1: array([[0, 2],
        [0, 4],
        [1, 3],
        [2, 0],
        [2, 2],
        [2, 3],
        [3, 1],
        [3, 2],
        [4, 1],
        [4, 5],
        [5, 1],
        [5, 3],
        [5, 5]]), 2: array([[0, 1],
        [0, 3],
        [0, 5],
        [1, 0],
        [1, 1],
        [1, 2],
        [1, 4],
        [1, 5],
        [2, 1],
        [2, 4],
        [3, 0],
        [3, 3],
        [4, 2],
        [4, 4],
        [5, 0]])}

In [28]:
test.map

array([[0, 2, 1, 2, 1, 2],
       [2, 2, 2, 1, 2, 2],
       [1, 2, 1, 1, 2, 0],
       [2, 1, 1, 2, 0, 0],
       [0, 1, 2, 0, 2, 1],
       [2, 1, 0, 1, 0, 1]])

In [33]:
x = [2,2]
x
len(np.where(np.isin(test.map[x[0]-1:x[0]+2, x[1]-1:x[1]+2],
                                [test.map[x[0],x[1]], 0]))[0])-1

4

In [ ]:
from matplotlib import animation


def animate_schelling(n):
    
    N = 100
    races = 2
    empty_ratio = .2
    similarity_threshold = .6
    n_iterations = 10

    schelling = Schelling(N, empty_ratio, similarity_threshold, races)

    schelling.update()

    fig, ax = plt.subplots(figsize=(6,6))
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
    ax.axis('off')
    ax.set(xlim=(-1, 1), ylim=(-1, 1))

    plt.imshow(test.map[1:test.N, 1:test.N], cmap=cmap);




    line, = ax.plot([], [], 'o-', lw=2)

    def init():
        line.set_data([], [])
        return line,

    def animate(i):
        line.set_data(x[i], y[i])
        return line,

    anim = animation.FuncAnimation(fig, animate, frames=len(t),
                                   interval=1000 * t.max() / len(t),
                                   blit=True, init_func=init)
    plt.close(fig)
    return anim